In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_csv("./all_df.csv")
df = df.dropna()
df.to_csv("./not_null_df.csv",encoding="utf-8-sig")

In [6]:
df = pd.read_csv("./not_null_df.csv")

In [7]:
df = df.drop(['Unnamed: 0'],axis=1)

In [8]:
del_index = df[df.skin_info=="복합성"].index
df = df.drop(index=del_index,axis=0)

In [9]:
user_to_replace_id = {original: idx for idx, original in enumerate(sorted(df.user_id.unique()))}
item_to_replace_id = {original: idx for idx, original in enumerate(sorted(df.item_id.unique()))}

In [10]:
df['item_to_replace_id'] = df["item_id"].map(lambda x:item_to_replace_id[x])

In [11]:
df['user_replace_id'] = df['user_id'].map(lambda x:user_to_replace_id[x])

In [12]:
result_df = df[['user_replace_id',"item_to_replace_id","user_rating","skin_type","skin_info","item_id","user_id","user_rating"]]

In [13]:
ratings = result_df

In [14]:
ratings.user_id.unique()

array(['lee0***', 'ljm3***', 'wngm****', ..., 'ehdw****', 'wlzu****',
       'kjcl**'], dtype=object)

In [15]:
user_dict = {}
for user_name,user_id in zip(ratings.user_id.unique(),ratings.user_replace_id.unique()):
    user_dict[user_name] = user_id

In [16]:
item_to_index_dict = {}
for item_name,item_id in zip(ratings.item_id.unique(),ratings.item_to_replace_id.unique()):
    item_to_index_dict[item_name] = item_id

In [17]:
index_to_item_id_dict = {}
for item_name,item_id in zip(ratings.item_id.unique(),ratings.item_to_replace_id.unique()):
    index_to_item_id_dict[item_id] = item_name

In [18]:
ratings

,user_replace_id,item_to_replace_id,user_rating,skin_type,skin_info,item_id,user_id,user_rating
0,9880,342,5.0,수분부족지성,모공,111950000047,lee0***,5.0
1,10206,342,5.0,수분부족지성,모공,111950000047,ljm3***,5.0
2,9880,342,5.0,수분부족지성,모공,111950000047,lee0***,5.0
3,10206,342,5.0,수분부족지성,모공,111950000047,ljm3***,5.0
4,17244,342,5.0,건성,건조함,111950000047,wngm****,5.0
...,...,...,...,...,...,...,...,...
99824,17002,8,5.0,극건성,탄력없음,5031000024,wind***,5.0
99825,17002,8,5.0,극건성,탄력없음,5031000024,wind***,5.0
99826,12713,8,5.0,지성,모공,5031000024,plus****,5.0
99827,15,3,5.0,복합성,트러블,1201000015,0109*******,5.0


In [19]:
def Item_base_cos_df(ratings):
    item_based = ratings.pivot_table('user_rating',index="item_to_replace_id",columns="user_replace_id")
    item_based.fillna(0,inplace=True)
    sim_rate = cosine_similarity(item_based,item_based)
    sim_rate_df = pd.DataFrame(
        data = sim_rate,
        index= item_based.index,
        columns = item_based.index
    )
    return sim_rate_df


In [20]:
def recommend(sim_rate_df,item_id,k):
    return sim_rate_df[item_id].sort_values(ascending=False)[1:k+1]

# 피부 타입, 피부 고민 별 item_based_cf 만들기

In [37]:
# 1. 건성, 민감성

In [21]:
def index_to_item_id(result_list):
    item_ids = []
    for result in result_list:
        item_id = index_to_item_id_dict[result]
        item_ids.append(int(item_id))
    return item_ids

In [22]:
def Item_based_filtering(df,parmas_skin_type,params_skin_info,item_name):
    item_id = item_to_index_dict[item_name] 
    skin_type=df[df["skin_type"]==parmas_skin_type]
    skin_info=skin_type[skin_type['skin_info']==params_skin_info]
    result = Item_base_cos_df(skin_info)
    result_list = recommend(result,item_id,10)
    item_id_list = index_to_item_id(result_list.index)
    return item_id_list
    

In [23]:
def Item_ALL_based_filtering(df,item_name):
    item_id = item_to_index_dict[item_name] 
    result = Item_base_cos_df(df)
    result_list = recommend(result,item_id,10)
    item_id_list = index_to_item_id(result_list.index)
    return item_id_list

In [24]:
df[df.item_to_replace_id==396]

,item_id,brand_name,category_id,item_name,price,avg_rating,user_id,user_rating,age_group,gender,skin_type,skin_info,review,item_to_replace_id,user_replace_id
64316,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,surf*******,5.0,4.0,남성,복합성,민감성,"괜찮은 제품이라 가격은 높지만, 계속 재구매하는 제품입니다.",396,15691
64317,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,0104*******,5.0,4.0,여성,수분부족지성,건조함,마스크 전엔 머리결이 거친데 도포 후 헹궈내면 머릿결이 보들보들 넘 좋아요,396,6
64318,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,lano***,5.0,4.0,여성,건성,건조함,부드럽게 헹궈져서 머릿결이 좋아지는 기분이 들어요,396,9799
64319,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,ujin****,5.0,4.0,여성,중성,민감성,항상 사용하고 있어요.\n\n40대가되니 탈모 고민이었는데 바로 눈에띄는 효과는 모...,396,16588
64320,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,sabl****,5.0,5.0,여성,건성,민감성,개인적으로 두피까지 다 사용할 수 있는 트리트먼트 제품을 선호해서 여러가지 써봤는데...,396,13904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64395,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,mint***,5.0,3.0,여성,건성,칙칙함,좋아요 요즘 탈모고민때문에 주문한건데 아윤채꺼야 워낙 믿을만한 브랜드이니까요\n가격...,396,11071
64396,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,kali***,4.0,3.0,여성,건성,건조함,80ml에 5만원이면 비싼가격인데 20퍼 할인에 다른종류 샘플도 써볼수있어서 평소 ...,396,8524
64397,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,indi******,5.0,5.0,여성,중성,탄력없음,스트레스밭을때나 염색잘못했을때\n머리빠짐이심살때 뿌리면 효과있어요\n끊적임 있어서...,396,7068
64398,765001000042,아윤채,CTG004,리밸런싱 마스크 200ml,40000.0,4.7,than*******,5.0,3.0,남성,지성,트러블,프리메라 제품을 쓰다 봐꿨는데 향도 좋고 진정효과도 있는것같아요,396,15960


In [33]:
skin_type=df[df["skin_type"]=="복합성"]
skin_info=skin_type[skin_type['skin_info']=="트러블"]
skin_info.item_id.unique() #복합성	트러블

array([111950000047, 111130000895, 111130000691, 111130000694,
       111380000257, 111170000097, 111130000542, 111650000231,
       111650000137, 111290000852, 110640000573, 111970001045,
       110640000058, 150100000017, 111380100213, 111170000292,
       111650000496, 111900000001, 111130000890, 111650000592,
       111650000483, 150100000054, 150100000091, 110640000614,
       110640100173, 150100000020, 110650001011, 111070101379,
       111130000873, 111830000013, 111070100435, 111900000002,
       111270000318, 111170000114, 111130000914, 111070000508,
       111170000802, 110640000222, 111970001212, 111650000141,
       110640000299, 111380000221, 111070000566, 111070100611,
       111970001223, 111170100139, 111970001213, 111070001080,
       111900000034, 111380000305, 111170000079, 111170000569,
       111380000088, 111170101199, 111170101128, 110640000170,
       111380000223, 110650000930, 110650000460, 110640000626,
       111170100172, 111130000657, 111170100160, 111170

In [25]:
result = Item_based_filtering(df,"수분부족지성","칙칙함",111310000214)
result

[177770000039,
 111270000167,
 110650000789,
 111070001519,
 110650100551,
 110100000032,
 177770000021,
 111170000079,
 111130000542,
 111170000273]

In [26]:
result =Item_ALL_based_filtering(df,111310000214)
result

[111310000163,
 111410000147,
 111650000195,
 110640000058,
 111290000268,
 111290000554,
 111270000142,
 111310000219,
 110640000222,
 111310000326]

In [42]:
df[df['item_id']==177770000039]

,item_id,brand_name,category_id,item_name,price,avg_rating,user_id,user_rating,age_group,gender,skin_type,skin_info,review,item_to_replace_id,user_replace_id
87614,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,rmsv**,5.0,2.0,남성,중성,모공,일단 제품의 길이가 길고 조금 두꺼운편이라 초보자분들이 손쉽게 사용할 수 있고 듀얼...,382,13655
87615,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,cure***,4.0,4.0,여성,복합성,모공,아이브러시를 따로 들고다니기 너무 귀찮아서 파우치속에서 넣고 마구쓸 브러시를 찾고있...,382,3130
87616,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,lde0***,5.0,3.0,여성,건성,건조함,"남성분들을 타깃으로 나온 브랜드같은데, 브러쉬가 활용도가 높아보여 구매했습니다! \...",382,9850
87617,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,0105*******,4.0,5.0,여성,수분부족지성,모공,브러쉬 모가 보드러워요! 브러쉬를 사용하면서 따갑다는 느낌을 받게하는 모가 많은데 ...,382,9
87618,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,sung*******,5.0,2.0,남성,지성,트러블,새 제품으로 오자마자 뜯어서 바로 사용해봤는데 매번 화장할때마다 여자친구가 쉐도우를...,382,15635
87619,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,mang***,5.0,3.0,여성,중성,민감성,듀얼 브러쉬라서 활용도가 좋을것 같아서 사봤어요. 비레디가 남성용 브랜드인것 같은데...,382,10665
87620,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,minh****,5.0,3.0,여성,중성,민감성,양쪽으로 브러쉬가 달려있어서 하나로 음영가능합니다. 가성비 좋아요..! 남성용이라서...,382,11039
87621,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,rla0***,5.0,5.0,여성,건성,민감성,안녕하세요 브러쉬 아주 잘쓰고있어요 가격이 좀 싸서 제대로 될까 걱정했는데 나름 만...,382,13579
87622,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,oxbr********,5.0,5.0,남성,복합성,건조함,휴대하거나 여행갈땐 듀얼 브러쉬가 확실히 좋습니다. 예상밖으로 좋은 품질이라 추가 ...,382,12348
87623,177770000039,비레디,CTG007,무드 업 듀얼 아이 섀도우 브러쉬 포 히어로즈,7500.0,4.7,mang***,5.0,3.0,여성,복합성,민감성,브러쉬 모가 너무 부드럽고 좋아요. 듀얼 브러쉬라서 한쪽은 베이스로 한쪽은 포인트 ...,382,10665
